In [3]:
import os

# Make sure models directory exists
os.makedirs("models", exist_ok=True)

# Now save
joblib.dump((vectorizer, df), "models/faq_tfidf.joblib")

NameError: name 'joblib' is not defined

In [13]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import make_pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report
# import joblib

# df = pd.read_csv("data/faq.csv")
# X = df['question']
# y = df['intent']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# pipe = make_pipeline(
#     TfidfVectorizer(ngram_range=(1,2), max_features=10000),
#     LogisticRegression(max_iter=2000)
# )
# pipe.fit(X_train, y_train)
# y_pred = pipe.predict(X_test)
# print(classification_report(y_test, y_pred))
# joblib.dump(pipe, "models/intent_baseline.joblib")

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

# Load CSV
df = pd.read_csv(r"C:\Users\Lenovo\Ai_farmer_query_based\data\faq2.csv")  # has 'questions' and 'answers'

# Fit TF-IDF on questions
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=10000)
X = vectorizer.fit_transform(df['questions'])

# Save model + data
joblib.dump((vectorizer, df), "models/faq_tfidf.joblib")

# Function to retrieve answer
def get_answer(query, top_k=1):
    query_vec = vectorizer.transform([query])
    sims = cosine_similarity(query_vec, X).flatten()
    top_idx = sims.argsort()[-top_k:][::-1]
    return [(df['questions'][i], df['answers'][i], sims[i]) for i in top_idx]

# Example
print(get_answer("What fertilizer should I use for wheat?"))



[('fertilizer dose for wheat', 'urea- 18kg, ssp- 33kg, mop- 8kg per bigha', 0.5461223972857809)]


In [15]:
# sketch: use Hugging Face Trainer API (install transformers, datasets)
# from datasets import Dataset
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# import pandas as pd

# df = pd.read_csv("data/faq.csv")
# label2id = {l:i for i,l in enumerate(sorted(df['intent'].unique()))}
# df['label'] = df['intent'].map(label2id)
# ds = Dataset.from_pandas(df[['question','label']])

# model_name = "distilbert-base-multilingual-cased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# def preprocess(ex):
#     return tokenizer(ex['question'], padding='max_length', truncation=True, max_length=128)

# ds = ds.map(preprocess, batched=True)
# ds = ds.train_test_split(test_size=0.1)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2id))

# args = TrainingArguments(output_dir="models/intent_tf", per_device_train_batch_size=16, num_train_epochs=3, evaluation_strategy="epoch")
# trainer = Trainer(model=model, args=args, train_dataset=ds['train'], eval_dataset=ds['test'])
# trainer.train()
# trainer.save_model("models/intent_tf")

from sentence_transformers import SentenceTransformer, util

# Load multilingual model
model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

# Encode questions
embeddings = model.encode(df['questions'].tolist(), convert_to_tensor=True)

def get_answer_embed(query, top_k=1):
    query_emb = model.encode(query, convert_to_tensor=True)
    sims = util.pytorch_cos_sim(query_emb, embeddings)[0]
    top_idx = sims.argsort(descending=True)[:top_k]

    seen = set()
    results = []
    for idx in top_idx:
        i = idx.item()
        if i not in seen:
            results.append((df['questions'][i], df['answers'][i], float(sims[i])))
            seen.add(i)
    return results
# Example
print(get_answer_embed("How to improve soil fertility naturally?"))


[('how to improve soil fertility of paddy field', 'use azola biofertilizer with paddy crops & also use psb culture 5g/kg of seed of rice', 0.8349844217300415)]


In [1]:
print(get_answer_embed("How to prevent farmer loss from heavy rainfall?"))

NameError: name 'get_answer_embed' is not defined